# OGG -> WAV

In [16]:
import numpy as np
import soundfile as sf
import typing

# import Vokaturi
import sys
sys.path.append("../api")
import Vokaturi
Vokaturi.load("../lib/Vokaturi_mac.so")

import audiotranscode
import emoji

In [20]:
def get_sample(path_ogg: str) -> tuple([np.array, int]):
    """
    :param path_ogg: path to .ogg file
    :return: samples of audio file and its sample rate
    """
    path_wav = path_ogg[:-3] + 'wav'
    
    at = audiotranscode.AudioTranscode()    
    at.transcode(path_ogg, path_wav)
    
    (samples, sample_rate) = sf.read(path_wav)

    return samples, sample_rate

def emotion_wrapper(path_ogg: str) -> tuple([bool, dict]):
    """
    :param path_ogg: path to .ogg file
    :return: indicator of validity and dictionary with detected
             emotions and its probabilities
    """
    (samples, sample_rate) = get_sample(path_ogg)
    buffer_length = len(samples)
    
    c_buffer = Vokaturi.SampleArrayC(buffer_length)
    c_buffer[:] = samples[:]

    voice = Vokaturi.Voice(sample_rate, buffer_length)

    # filling Voice with `samples`
    voice.fill(buffer_length, c_buffer)
    quality = Vokaturi.Quality()
    emotionProbabilities = Vokaturi.EmotionProbabilities()
    voice.extract(quality, emotionProbabilities)
    voice.destroy()

    quality.num_frames_analyzed
    
    emo_dict = {"neutrality": emotionProbabilities.neutrality,
                "happiness": emotionProbabilities.happiness,
                "sadness": emotionProbabilities.sadness,
                "anger": emotionProbabilities.anger,
                "fear": emotionProbabilities.fear}

    return quality.valid, emo_dict

In [57]:
def with_emoji(emo_dict: dict, mapping: dict) -> list:
    """
    :param emo_dict:
    :param mapping:
    :return:
    """
    text = []

    for emo, prob in emo_dict.items():
        text.append(emoji.emojize('{emo_str}: {prob_str:.3f}'.format(emo_str=mapping[emo],
                                                                 prob_str=prob), use_aliases=True))
        
    return text

# Work with test.wav

In [21]:
if quality.valid:
    print(emoji.emojize(':neutral_face: %.3f', use_aliases=True) % emotionProbabilities.neutrality)
    print(emoji.emojize(":smile: %.3f", use_aliases=True) % emotionProbabilities.happiness)
    print(emoji.emojize(":pensive: %.3f", use_aliases=True) % emotionProbabilities.sadness)
    print(emoji.emojize(":astonished: %.3f", use_aliases=True) % emotionProbabilities.anger)
    print(emoji.emojize(":scream: %.3f", use_aliases=True) % emotionProbabilities.fear)
else:
    print ("Not enough sonorancy to determine emotions")

😐 0.001
😄 0.000
😔 0.003
😲 0.996
😱 0.000


In [22]:
emotion_wrapper('wow_sure.ogg')

(1,
 {'anger': 0.9957252540976576,
  'fear': 3.192306066297615e-12,
  'happiness': 0.0004755844285208308,
  'neutrality': 0.000608280949175865,
  'sadness': 0.0031908805214534344})

In [38]:
emo_mapping = {
    'anger': ':astonished:',
    'fear': ':scream:',
    'happiness': ':smile:',
    'neutrality': ':neutral_face:',
    'sadness': ':pensive:'
}

In [54]:
valid, emo_dict = emotion_wrapper('wow_sure.ogg')

1

In [56]:
emo_dict

{'anger': 0.9957252540976576,
 'fear': 3.192306066297615e-12,
 'happiness': 0.0004755844285208308,
 'neutrality': 0.000608280949175865,
 'sadness': 0.0031908805214534344}

In [71]:
if valid:
    print('\n'.join(with_emoji(emo_dict, emo_mapping)))
else:
    err_message = emoji.emojize("¯\_(ツ)_/¯", use_aliases=True)
    print(err_message)

😐: 0.001
😄: 0.000
😔: 0.003
😲: 0.996
😱: 0.000


In [75]:
emo_dict

{'anger': 0.9957252540976576,
 'fear': 3.192306066297615e-12,
 'happiness': 0.0004755844285208308,
 'neutrality': 0.000608280949175865,
 'sadness': 0.0031908805214534344}

In [76]:
testdict = {'a': 1, 'b': 2}

In [83]:
dict(zip(testdict.values(), testdict.keys()))

{1: 'a', 2: 'b'}